### [color shape number] splits were decided based on Counters

Jul 21 Update: What colour (), What's the colour (1), What is the colour (5), Which colour (3) previously sneaked into "Others". They've been brought back to "color". Those with 'blue' or 'purple' as answers are put into test set. Qcate statistics in this notebook may be different from the previous ones for Qcate assignments and splits.

### [YesNo choose] splits were decided based on model performance (fail 3/3 times)
### [Others] split will be decided randomly, with 19.5% falling into ood. [Others] is considered intrinsically hard

In [2]:
import json
import random
import copy
import numpy as np
import pickle
from collections import Counter
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

In [5]:
# Sanity check after img_dataset_J_Qcate.json is fixed
dataset_J_under_processing = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_data_J_color_shape_number_have_split.json", "r"))
print(Counter([dataset_J_under_processing[k]['Qcate'] for k in dataset_J_under_processing]))
print(len(set([dataset_J_under_processing[k]['Guid'] for k in dataset_J_under_processing])))
print(Counter([len(dataset_J_under_processing[j]['GoldIds']) for j in dataset_J_under_processing]))
print([k for k in range(25474) if str(k) not in dataset_J_under_processing])

Counter({'YesNo': 8432, 'Others': 6748, 'choose': 5240, 'number': 2341, 'color': 2044, 'shape': 662})
25467
Counter({1: 14337, 2: 11130})
[233, 338, 443, 1233, 1234, 2141, 2361]


In [7]:
YesNo_choose_ood_guids = pickle.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/YesNo_choose_ood_guid.pkl", "rb"))
print(len(YesNo_choose_ood_guids))

2129


In [21]:
for k in dataset_J_under_processing:
    if dataset_J_under_processing[k]['Qcate'] in ['YesNo', 'choose']:
        if dataset_J_under_processing[k]['Guid'] in YesNo_choose_ood_guids:
            dataset_J_under_processing[k]["split"] = 'ood_test'
        elif random.random() < 0.77:
            dataset_J_under_processing[k]["split"] = 'train'
        elif random.random() < 0.5:
            dataset_J_under_processing[k]["split"] = 'val'
        else:
            dataset_J_under_processing[k]["split"] = 'ind_test'
    elif dataset_J_under_processing[k]['Qcate'] == 'Others':
        if random.random() < 0.195:
            dataset_J_under_processing[k]["split"] = 'ood_test'
        elif random.random() < 0.75:
            dataset_J_under_processing[k]["split"] = 'train'
        elif random.random() < 0.5:
            dataset_J_under_processing[k]["split"] = 'val'
        else:
            dataset_J_under_processing[k]["split"] = 'ind_test'
    elif dataset_J_under_processing[k]['Qcate'] in ['number', 'color', 'shape']:
        pass
    else:
        print("wtf???")

In [23]:
# json.dump(dataset_J_under_processing, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/dataset_J_split_finalized0629.json", "w"), indent=4)

# This version was finalized on Jun 29

### After Jul 21 Update

In [2]:
dataset_J_under_processing = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/dataset_J_split_update0721.json", "r"))
print(len(set([dataset_J_under_processing[k]['Guid'] for k in dataset_J_under_processing])))
print(Counter([len(dataset_J_under_processing[j]['GoldIds']) for j in dataset_J_under_processing]))
print([k for k in range(25474) if str(k) not in dataset_J_under_processing])
print(Counter([dataset_J_under_processing[k]['Qcate'] for k in dataset_J_under_processing]))

25467
Counter({1: 14337, 2: 11130})
[233, 338, 443, 1233, 1234, 2141, 2361]


In [7]:
for k in dataset_J_under_processing:
    if any([x in dataset_J_under_processing[k]['Q'].lower() for x in ['which colour', 'what colour', "what's the colour", 'what is the colour']]):
        assert dataset_J_under_processing[k]['Qcate'] == 'color'
        if any([b in dataset_J_under_processing[k]['A'].lower() for b in ['blue', 'purple']]):
            assert dataset_J_under_processing[k]['split'] == 'ood_test'
        else:
            assert not dataset_J_under_processing[k]['split'] == 'ood_test'
print(Counter([dataset_J_under_processing[k]['Qcate'] for k in dataset_J_under_processing]))

Counter({'YesNo': 8432, 'Others': 6721, 'choose': 5240, 'number': 2341, 'color': 2071, 'shape': 662})


In [8]:
index = ['YesNo', 'choose', 'Others', 'number', 'color', 'shape', 'Total']
columns = ['ood_test', 'train', 'val', 'ind_test']
split_df = pd.DataFrame(index = index, columns = columns)
for i in index:
    for col in columns:
        if i == 'Total':
            split_df.loc[i, col] = len([k for k in dataset_J_under_processing if dataset_J_under_processing[k]['split'] == col])
        else:
            split_df.loc[i, col] = len([k for k in dataset_J_under_processing if dataset_J_under_processing[k]['Qcate'] == i and dataset_J_under_processing[k]['split'] == col])
split_df.head(7)

,ood_test,train,val,ind_test
YesNo,1109,5670,828,825
choose,1020,3210,502,508
Others,1302,4101,669,649
number,221,1626,260,234
color,239,1426,180,226
shape,97,425,74,66
Total,3988,16458,2513,2508


In [14]:
split_ratio_df = pd.DataFrame(index = index, columns = columns)
denos = [8432, 5240, 6721, 2341, 2071, 662, 25467]
assert sum(denos) == 25467*2
index2total = dict(zip(index, denos))
for i in index:
    for col in columns:
        split_ratio_df.loc[i, col] = split_df.loc[i, col] / index2total[i]
split_ratio_df.head(7)

,ood_test,train,val,ind_test
YesNo,0.13,0.67,0.10,0.10
choose,0.19,0.61,0.10,0.10
Others,0.19,0.61,0.10,0.10
number,0.09,0.69,0.11,0.10
color,0.12,0.69,0.09,0.11
shape,0.15,0.64,0.11,0.10
Total,0.16,0.65,0.10,0.10


In [12]:
dataset_J_split_finalized0629 = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/dataset_J_split_update0721.json", "r"))
print(Counter([dataset_J_split_finalized0629[k]['Qcate'] for k in dataset_J_split_finalized0629]))
print(len(set([dataset_J_split_finalized0629[k]['Guid'] for k in dataset_J_split_finalized0629])))
print(Counter([len(dataset_J_split_finalized0629[j]['GoldIds']) for j in dataset_J_split_finalized0629]))
print(Counter([dataset_J_split_finalized0629[j]['split'] for j in dataset_J_split_finalized0629]))
print([k for k in range(25474) if str(k) not in dataset_J_split_finalized0629])

Counter({'YesNo': 8432, 'Others': 6721, 'choose': 5240, 'number': 2341, 'color': 2071, 'shape': 662})
25467
Counter({1: 14337, 2: 11130})
Counter({'train': 16458, 'ood_test': 3988, 'val': 2513, 'ind_test': 2508})
[233, 338, 443, 1233, 1234, 2141, 2361]


In [15]:
# Check if Qtype (['Multihop', 'Singlehop']) distribution was skewed in ood_test
index = ['YesNo', 'choose', 'Others', 'number', 'color', 'shape', 'Total']
columns = ['%Multihop in ood_test', '%Multihop in train/val/ind_test', '%Multihop in Total']
Qtype_df = pd.DataFrame(index = index, columns = columns)
for i in index:
    if i == 'Total':
        Qtype_df.loc[i, '%Multihop in ood_test'] = 100 * len([k for k in dataset_J_split_finalized0629 if dataset_J_under_processing[k]['Qtype'] == 'img-Multihop' and dataset_J_under_processing[k]['split'] == 'ood_test']) / len([k for k in dataset_J_split_finalized0629 if dataset_J_under_processing[k]['split'] == 'ood_test'])
        Qtype_df.loc[i, '%Multihop in train/val/ind_test'] = 100 * len([k for k in dataset_J_split_finalized0629 if dataset_J_under_processing[k]['Qtype'] == 'img-Multihop' and (not dataset_J_under_processing[k]['split'] == 'ood_test')]) / len([k for k in dataset_J_split_finalized0629 if not dataset_J_under_processing[k]['split'] == 'ood_test'])
        Qtype_df.loc[i, '%Multihop in Total'] = 100 * len([k for k in dataset_J_split_finalized0629 if dataset_J_under_processing[k]['Qtype'] == 'img-Multihop']) / len([k for k in dataset_J_split_finalized0629])
    else:
        Qtype_df.loc[i, '%Multihop in ood_test'] = 100 * len([k for k in dataset_J_split_finalized0629 if dataset_J_split_finalized0629[k]['Qcate'] == i and dataset_J_under_processing[k]['Qtype'] == 'img-Multihop' and dataset_J_under_processing[k]['split'] == 'ood_test']) / len([k for k in dataset_J_split_finalized0629 if dataset_J_split_finalized0629[k]['Qcate'] == i and dataset_J_under_processing[k]['split'] == 'ood_test'])
        Qtype_df.loc[i, '%Multihop in train/val/ind_test'] = 100 * len([k for k in dataset_J_split_finalized0629 if dataset_J_split_finalized0629[k]['Qcate'] == i and dataset_J_under_processing[k]['Qtype'] == 'img-Multihop' and (not dataset_J_under_processing[k]['split'] == 'ood_test')]) / len([k for k in dataset_J_split_finalized0629 if dataset_J_split_finalized0629[k]['Qcate'] == i and (not dataset_J_under_processing[k]['split'] == 'ood_test')])
        Qtype_df.loc[i, '%Multihop in Total'] = 100 * len([k for k in dataset_J_split_finalized0629 if dataset_J_split_finalized0629[k]['Qcate'] == i and dataset_J_under_processing[k]['Qtype'] == 'img-Multihop']) / len([k for k in dataset_J_split_finalized0629 if dataset_J_split_finalized0629[k]['Qcate'] == i])
Qtype_df.head(7)

,%Multihop in ood_test,%Multihop in train/val/ind_test,%Multihop in Total
YesNo,52.39,46.66,47.41
choose,55.29,65.14,63.23
Others,21.04,23.45,22.99
number,3.62,5.14,5.00
color,16.32,20.31,19.85
shape,8.25,9.20,9.06
Total,36.96,37.11,37.08


In [3]:
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/dataset_J_split_update0721.json", "r"))
print(len(set([img_dataset[k]['Guid'] for k in img_dataset])))
print(Counter([len(img_dataset[j]['GoldIds']) for j in img_dataset]))
print([k for k in range(25474) if str(k) not in img_dataset])
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset]))

25467
Counter({1: 14337, 2: 11130})
[233, 338, 443, 1233, 1234, 2141, 2361]
Counter({'YesNo': 8432, 'Others': 6721, 'choose': 5240, 'number': 2341, 'color': 2071, 'shape': 662})
